## Prepare dataset

In [ ]:
DATASET_NAME = "IXI"
DATA_PATH = "/data/falcetta/brain_data/IXI/volumes"
OUTPUT_PATH = "/data/falcetta/brain_data/IXIJ/processed"

#these functions are to define for each new dataset, unless fixing a rule
#they allow to define a clear association between the data path of a brain image and the corresponding brain/vessel mask
#the brain mask is not mandatory, if not present just return None
import utils
def img_path_to_brain_path(img_path):
    out_path =img_path.replace("volumes/","masks/").replace(".nii.gz","_mask.nii.gz")
    return out_path
def img_path_to_vessel_path(img_path):
    out_path =img_path.replace("volumes/","vessels/").replace(".nii.gz","_vessel_mask2.nii.gz")
    return out_path
def img_path_to_weight_path(img_path):
    out_path =img_path.replace(".nii.gz","_weight.nii.gz")
    return out_path

utils.img_path_to_brain_path = img_path_to_brain_path
utils.img_path_to_vessel_path = img_path_to_vessel_path
utils.img_path_to_weight_path = img_path_to_weight_path

#this regex works as a filter in case you have unnecessary .nii files in your dataset folder
PATH_RULE = "" #"^.*MRA.nii.gz$"

#if you have some outliers you want to discard, you can specify it in the list below
OUTLIERs = []

#if you need to preprocess also images with no vessel masks set the following flag to False
VESSELs_REQUIRED = False

#this parameter is to unify the orientation of our images
#set it True if you notice in the examples shown below that the nose is oriented downward and not upward
utils.do_flip = False

### Extract paths

In [ ]:
import os

from utils import search_nii, extract_paths, load_info_from_checkpoint

In [ ]:
if not os.path.isfile(f"info_{DATASET_NAME}.pkl"):
    info = {
        "train": extract_paths(path_rule=PATH_RULE),
        "val": None,
        "test": None,
    }
    search_nii(DATA_PATH, info["train"])
else:
    info = load_info_from_checkpoint(f"info_{DATASET_NAME}.pkl")

### Display examples

In [ ]:
import random

from utils import print_bold, load_and_display_middle_slice

In [ ]:
print_bold("[Example 01]")
random.seed(0)
load_and_display_middle_slice(random.choice(info["train"].paths), display_header=True)

print_bold("[Example 02]")
random.seed(1)
load_and_display_middle_slice(random.choice(info["train"].paths), axis=[0,1,2], display_header=True)

### Split train/val/test

In [ ]:
from utils import is_medical_volume

In total, there are

In [ ]:
IMGs = sum([
    info["train"].paths,
    info["val"].paths if info["val"] is not None else [],
    info["test"].paths if info["test"] is not None else []
], [])

len(IMGs)

volumes. First, we discard the outliers:

In [ ]:
IMGs = [img_path for img_path in IMGs if os.path.basename(img_path) not in OUTLIERs]

len(IMGs)

We have vessel annotations for

In [ ]:
HAVE_VESSELs = [
    img_path for img_path in IMGs if is_medical_volume(img_path_to_vessel_path(img_path))
]

len(HAVE_VESSELs)

of them. With a chosen ratio of 70-15-15, we select

In [ ]:
COUNT_TEST = 0 # len(HAVE_VESSELs if VESSELs_REQUIRED else IMGs) * 15 // 100
COUNT_TEST

images to be part of the validation/testing set.

In case brain masks are available, we collect them too.

In [ ]:
HAVE_BRAINs = [
    img_path for img_path in IMGs if is_medical_volume(img_path_to_brain_path(img_path))
]

len(HAVE_BRAINs)

Finally, we randomly split our dataset. 

In [ ]:
random.seed(0)

VAL_IMGs = random.sample(sorted(HAVE_VESSELs), COUNT_TEST)

print("val: ", [os.path.basename(img) for img in VAL_IMGs])

random.seed(1)

TEST_IMGs = random.sample(sorted([img for img in HAVE_VESSELs if img not in VAL_IMGs]), COUNT_TEST)

print("test: ", [os.path.basename(img) for img in TEST_IMGs])

TRAIN_IMGs = [img for img in (HAVE_VESSELs if VESSELs_REQUIRED else IMGs) if img not in VAL_IMGs and img not in TEST_IMGs]

print("train: ", [os.path.basename(img) for img in TRAIN_IMGs])

In [ ]:
# # PUT THE FIRST 3 OF THE TRAINING SET INTO THE TEST SET (AND REMOVE THEM FROM THE TRAINING SET)
# TEST_IMGs += TRAIN_IMGs[:3]
# TRAIN_IMGs = TRAIN_IMGs[3:]

# print(f"train (LEN {len(TRAIN_IMGs)}): ", [os.path.basename(img) for img in TRAIN_IMGs])
# print(f"test: (LEN {len(TEST_IMGs)}): ", [os.path.basename(img) for img in TEST_IMGs])

In [ ]:
import pickle


with open(f"info_{DATASET_NAME}.pkl", "wb") as file:
    pickle.dump({
        k: info[k].__dict__ if info[k] is not None else None for k in info
    }, file)

### Extract spacings and shapes

In [ ]:
from utils import load_info_from_checkpoint

info = load_info_from_checkpoint(f"info_{DATASET_NAME}.pkl")

In [ ]:
from utils import extract_info_and_masks, loop_nii, display_info, get_target_spacing

In [ ]:
info = {
    "train": extract_info_and_masks(path_rule=PATH_RULE),
    "val": extract_info_and_masks(path_rule=PATH_RULE),
    "test": extract_info_and_masks(path_rule=PATH_RULE),
}

loop_nii(TRAIN_IMGs, info["train"])
loop_nii(VAL_IMGs, info["val"])
loop_nii(TEST_IMGs, info["test"])

In [ ]:
display_info(info["train"], info["train"])#info["val"])

In [ ]:
SPACING = get_target_spacing(
    info["train"].spacings + info["val"].spacings,
    info["train"].shapesAfterCropping() + info["val"].shapesAfterCropping()
)

SPACING

In [ ]:
import pickle

with open(f"info_{DATASET_NAME}.pkl", "wb") as file:
    pickle.dump({
        k: info[k].__dict__ if info[k] is not None else None for k in info
    }, file)

### Crop, Metadata, Resize, Empty Slices Removal, Standardization

In [ ]:
from utils import load_info_from_checkpoint

info = load_info_from_checkpoint(f"info_{DATASET_NAME}.pkl")

In [ ]:
import os

from utils import print_bold, preprocessing_loop

In [ ]:
os.getcwd()

In [ ]:
print_bold("Training")
preprocessing_loop(
    info["train"],
    os.path.join(OUTPUT_PATH,"numpy", "train"),
    target_spacing=SPACING
)

# print_bold("Validation")
# preprocessing_loop(
#     info["val"],
#     os.path.join(OUTPUT_PATH,"numpy", "val"),
#     target_spacing=SPACING
# )

# print_bold("Testing")
# preprocessing_loop(
#      info["test"],
#      os.path.join(OUTPUT_PATH,"numpy", "test"),
#      target_spacing=SPACING
#  )

In [ ]:
import pickle

with open(f"info_{DATASET_NAME}.pkl", "wb") as file:
    pickle.dump({
        k: info[k].__dict__ if info[k] is not None else None for k in info
    }, file)

In [ ]:
import os
from pathlib import Path
import shutil 

def rename_dataset(DATASET_PATH_RENAMED, OUTPUT_PATH_RENAMED):
    # DELETE OUTPUT FOLDER IF EXISTS
    if os.path.exists(OUTPUT_PATH_RENAMED):
        shutil.rmtree(OUTPUT_PATH_RENAMED)
    
    Path(OUTPUT_PATH_RENAMED).mkdir(parents=True, exist_ok=True)

    print("Number of subjects: ", len(os.listdir(DATASET_PATH_RENAMED))/3)
    os.listdir(DATASET_PATH_RENAMED)

    dict_id = {}
    counter = 0

    for img in os.listdir(DATASET_PATH_RENAMED):
        patient_id = img.split("_")[0]
        if patient_id not in dict_id:
            counter += 1
            dict_id[patient_id] = str(counter).zfill(3)
        # copy to output folder and rename\
        print("Renaming ", img, " to ", dict_id[patient_id] + "_" + img.split("_")[-1])
        shutil.copy(os.path.join(DATASET_PATH_RENAMED, img), os.path.join(OUTPUT_PATH_RENAMED, dict_id[patient_id] + "_" + img))
        #os.rename(os.path.join(DATASET_PATH_RENAMED, img), os.path.join(OUTPUT_PATH_RENAMED, dict_id[patient_id] + "_" + img))

rename_dataset(os.path.join(OUTPUT_PATH, "numpy", "train"), os.path.join(OUTPUT_PATH, "numpy_renamed", "train"))
#rename_dataset(os.path.join(OUTPUT_PATH, "numpy", "test"), os.path.join(OUTPUT_PATH, "numpy_renamed", "test"))